In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [22]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [23]:
df["pickup_hour"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 87620 entries, 0 to 87619
Series name: pickup_hour
Non-Null Count  Dtype         
--------------  -----         
87620 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 684.7 KB


In [24]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 10, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(63700, 674)
(63700,)
(23920, 674)
(23920,)


In [25]:
def average_rides_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    last_4_weeks_columns = [
            f"rides_t-{7*24}",  # 1 week ago
            f"rides_t-{14*24}", # 2 weeks ago
            f"rides_t-{21*24}", # 3 weeks ago
            f"rides_t-{28*24}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
    for col in last_4_weeks_columns:
        if col not in X.columns:
            raise ValueError(f"Missing required column: {col}")

    # Calculate the average of the last 4 weeks
    X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)

    return X

from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_4_weeks = FunctionTransformer(
    average_rides_last_4_weeks, validate=False
)

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_["hour"] = X_["pickup_hour"].dt.hour
        X_["day_of_week"] = X_["pickup_hour"].dt.dayofweek

        return X_.drop(columns=["pickup_hour", "pickup_location_id"])

add_temporal_features = TemporalFeatureEngineer()


In [27]:
param_distributions = {
    "lgbmregressor__num_leaves": [2, 50, 70, 256],
    "lgbmregressor__max_depth": [-1, 10, 20, 30],
    "lgbmregressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "lgbmregressor__n_estimators": [100, 200, 500, 1000],
    "lgbmregressor__min_child_samples": [10, 20, 30, 50],
    "lgbmregressor__subsample": [0.6, 0.8, 1.0],
    "lgbmregressor__colsample_bytree": [0.6, 0.8, 1.0],
    "lgbmregressor__reg_alpha": [0, 0.1, 0.5, 1.0],
    "lgbmregressor__reg_lambda": [0, 0.1, 0.5, 1.0],
    "lgbmregressor__feature_fraction": [0.6, 0.7, 0.8, 0.9, 1.0],
    "lgbmregressor__bagging_fraction": [0.6, 0.7, 0.8, 0.9, 1.0],
    "lgbmregressor__bagging_freq": [1, 5, 10]
}


In [28]:
import lightgbm as lgb

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor(verbose=-1)
)

In [29]:
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('functiontransformer',
                                              FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000018FA8FE6020>)),
                                             ('temporalfeatureengineer',
                                              TemporalFeatureEngineer()),
                                             ('lgbmregressor',
                                              LGBMRegressor(verbose=-1))]),
                   n_jobs=-1,
                   param_distributions={'lgbmregressor__bagging_fraction': [0.6,
                                                                            0.7,
                                                                            0.8,
                                                                            0.9,
                                                                            1.0],
                                        'lgbmr...
                                        'lgbmregressor__max_depth': [-1, 10, 20,
                                                                     30],
                                        'lgbmregressor__min_child_samples': [10,
                                                                             20,
                                                                             30,
                                                                             50],
                                        'lgbmregressor__n_estimators': [100,
                                                                        200,
                                                                        500,
                                                                        1000],
                                        'lgbmregressor__num_leaves': [2, 50, 70,
                                                                      256],
                                        'lgbmregressor__reg_alpha': [0, 0.1,
                                                                     0.5, 1.0],
                                        'lgbmregressor__reg_lambda': [0, 0.1,
                                                                      0.5,
                                                                      1.0],
                                        'lgbmregressor__subsample': [0.6, 0.8,
                                                                     1.0]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [30]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('functiontransformer',
                                              FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000018FA8FE6020>)),
                                             ('temporalfeatureengineer',
                                              TemporalFeatureEngineer()),
                                             ('lgbmregressor',
                                              LGBMRegressor(verbose=-1))]),
                   n_jobs=-1,
                   param_distributions={'lgbmregressor__bagging_fraction': [0.6,
                                                                            0.7,
                                                                            0.8,
                                                                            0.9,
                                                                            1.0],
                                        'lgbmr...
                                        'lgbmregressor__max_depth': [-1, 10, 20,
                                                                     30],
                                        'lgbmregressor__min_child_samples': [10,
                                                                             20,
                                                                             30,
                                                                             50],
                                        'lgbmregressor__n_estimators': [100,
                                                                        200,
                                                                        500,
                                                                        1000],
                                        'lgbmregressor__num_leaves': [2, 50, 70,
                                                                      256],
                                        'lgbmregressor__reg_alpha': [0, 0.1,
                                                                     0.5, 1.0],
                                        'lgbmregressor__reg_lambda': [0, 0.1,
                                                                      0.5,
                                                                      1.0],
                                        'lgbmregressor__subsample': [0.6, 0.8,
                                                                     1.0]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [31]:
best_model = random_search.best_estimator_


In [32]:
y_pred = best_model.predict(X_test)

In [33]:
test_mae = mean_absolute_error(y_test, y_pred)
print(f"Test MAE: {test_mae:.4f}")

Test MAE: 2.9623


| **Hyperparameter**       | **Description**                                                                                                                                                                                                 | **Impact**                                                                                                                                                                                                                     |
|---------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `num_leaves`             | Maximum number of leaves in one tree. Larger values increase model complexity and accuracy but may lead to overfitting.                                                                                         | High: Directly controls the complexity of the model. Larger values can improve accuracy but increase the risk of overfitting.                                                                                                  |
| `max_depth`              | Maximum depth of a tree. Limits the depth of the tree to prevent overfitting. Use `-1` for no limit.                                                                                                           | High: Limits the model's ability to capture complex patterns. Shallower trees reduce overfitting but may underfit.                                                                                                             |
| `learning_rate`          | Step size for updating weights. Smaller values make training slower but improve generalization.                                                                                                                | High: Affects convergence speed and generalization. Lower values often require more iterations (`n_estimators`).                                                                                                               |
| `n_estimators`           | Number of boosting iterations (trees). Higher values improve accuracy but increase training time.                                                                                                              | High: Controls the number of trees in the ensemble. Too many trees can lead to overfitting if `learning_rate` is not adjusted.                                                                                                |
| `feature_fraction`       | Fraction of features (columns) to randomly select for each tree. Helps prevent overfitting.                                                                                                                    | Medium-High: Reduces overfitting and speeds up training. Lower values may lead to underfitting.                                                                                                                               |
| `bagging_fraction`       | Fraction of data (rows) to randomly select for each iteration. Works with `bagging_freq`.                                                                                                                       | Medium-High: Reduces overfitting and improves generalization. Lower values may lead to underfitting.                                                                                                                           |
| `bagging_freq`           | Frequency of bagging. For example, `5` means bagging is performed every 5 iterations.                                                                                                                          | Medium: Works with `bagging_fraction` to control how often bagging is applied.                                                                                                                                                |
| `min_child_samples`      | Minimum number of data points required in a leaf. Larger values prevent overfitting by limiting leaf size.                                                                                                     | Medium: Controls overfitting by limiting the size of leaf nodes. Higher values may lead to underfitting.                                                                                                                       |
| `colsample_bytree`       | Alias for `feature_fraction`. Fraction of features to randomly select for each tree.                                                                                                                            | Medium: Similar to `feature_fraction`, reduces overfitting and speeds up training.                                                                                                                                            |
| `subsample`              | Alias for `bagging_fraction`. Fraction of data to randomly select for each iteration.                                                                                                                           | Medium: Similar to `bagging_fraction`, reduces overfitting and improves generalization.                                                                                                                                       |
| `reg_alpha`              | L1 regularization term on weights. Adds a penalty for large coefficients to reduce overfitting.                                                                                                                | Medium: Helps reduce overfitting, especially in high-dimensional data.                                                                                                                                                        |
| `reg_lambda`             | L2 regularization term on weights. Adds a penalty for large coefficients to reduce overfitting.                                                                                                                | Medium: Similar to `reg_alpha`, but penalizes squared coefficients.                                                                                                                                                           |
| `max_bin`                | Maximum number of bins for discretizing continuous features. Higher values improve accuracy but increase training time.                                                                                         | Medium: Affects how continuous features are bucketed. Higher values improve precision but increase computational cost.                                                                                                         |
| `min_split_gain`         | Minimum gain required to split a node. Higher values prevent splitting nodes with low information gain.                                                                                                         | Medium-Low: Helps control overfitting by limiting unnecessary splits.                                                                                                                                                         |
| `boosting_type`          | Type of boosting algorithm. Options: `gbdt` (default), `dart`, `goss`.                                                                                                                                          | Medium-Low: Affects the boosting strategy. `dart` and `goss` are alternatives to `gbdt` for specific use cases.                                                                                                               |
| `objective`              | Objective function to optimize. Common options: `regression`, `regression_l1`, `huber`, `fair`.                                                                                                                | Medium-Low: Determines the loss function. Impacts how the model optimizes predictions.                                                                                                                                         |
| `verbosity`              | Controls the level of logging. Higher values provide more detailed logs.                                                                                                                                       | Low: Does not affect model performance but helps with debugging.                                                                                                                                                              |
| `random_state`           | Seed for reproducibility. Ensures consistent results across runs.                                                                                                                                               | Low: Does not affect model performance but ensures reproducibility.                                                                                                                                                           |
| `early_stopping_round`   | Stops training if validation score does not improve for a specified number of rounds.                                                                                                                           | Low: Helps save time during training but does not directly affect model performance.                                                                                                                                           |

In [35]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(best_model, X_test, "LGBMRegressor with FE, HT", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.
INFO:src.experiment_utils:Experiment set to: LGBMRegressor with FE, HT
INFO:src.experiment_utils:Logged mean_absolute_error: 2.962337397709399
c:\Users\vidyu\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:

🏃 View run LGBM with Hyperparameter Tuning at: https://dagshub.com/vidyuthkrishna03/nyc-taxi-rides-prediction.mlflow/#/experiments/8/runs/c48aa1c068d14ba6aafe5b635d8ab747
🧪 View experiment at: https://dagshub.com/vidyuthkrishna03/nyc-taxi-rides-prediction.mlflow/#/experiments/8
